In [33]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tfr
import numpy as np
import matplotlib.pyplot as plt
from glob import glob


In [34]:
TRAIN_DIR = "/kaggle/input/stanford-cars-dataset/cars_train"
TEST_DIR = "/kaggle/input/stanford-cars-dataset/cars_test"
cars_train_dir = os.path.join(TRAIN_DIR, "cars_train")
cars_test_dir = os.path.join(TEST_DIR, "cars_test")

In [35]:
print("Cars Train: ", len(os.listdir(cars_train_dir)))
print("Cars Test: ", len(os.listdir(cars_test_dir)))


cars_train_files = os.listdir(cars_train_dir)
print(cars_train_files[:5])

cars_test_files = os.listdir(cars_test_dir)
print(cars_test_files[:5])

Cars Train:  8144
Cars Test:  8041
['05938.jpg', '06122.jpg', '04168.jpg', '02371.jpg', '04377.jpg']
['05938.jpg', '06122.jpg', '04168.jpg', '02371.jpg', '04377.jpg']


In [36]:
import scipy.io as sio
import pandas as pd

# Replace 'path_to_file' with the actual path to your .mat file.
mat_data = sio.loadmat('/kaggle/input/stanford-cars-dataset/cars_annos.mat')
print(mat_data.keys())

data = mat_data['annotations']

# Extract individual arrays from the 'data' variable
image_paths = data['relative_im_path'].squeeze()   # Assuming 'relative_im_path' is the field name for image paths
x_min = data['bbox_x1'].squeeze()                  # Assuming 'bbox_x1' is the field name for x_min
y_min = data['bbox_y1'].squeeze()                  # Assuming 'bbox_y1' is the field name for y_min
x_max = data['bbox_x2'].squeeze()                  # Assuming 'bbox_x2' is the field name for x_max
y_max = data['bbox_y2'].squeeze()                  # Assuming 'bbox_y2' is the field name for y_max

# Create a DataFrame from the extracted arrays
annotations_df = pd.DataFrame({
    'image_path': image_paths,
    'x_min': x_min,
    'y_min': y_min,
    'x_max': x_max,
    'y_max': y_max
})


dict_keys(['__header__', '__version__', '__globals__', 'annotations', 'class_names'])


In [37]:
annotations_df.head()


,image_path,x_min,y_min,x_max,y_max
0,[car_ims/000001.jpg],[[112]],[[7]],[[853]],[[717]]
1,[car_ims/000002.jpg],[[48]],[[24]],[[441]],[[202]]
2,[car_ims/000003.jpg],[[7]],[[4]],[[277]],[[180]]
3,[car_ims/000004.jpg],[[33]],[[50]],[[197]],[[150]]
4,[car_ims/000005.jpg],[[5]],[[8]],[[83]],[[58]]


In [38]:
# Convert the bounding box coordinates to single values
annotations_df['x_min'] = annotations_df['x_min'].astype(int)
annotations_df['y_min'] = annotations_df['y_min'].astype(int)
annotations_df['x_max'] = annotations_df['x_max'].astype(int)
annotations_df['y_max'] = annotations_df['y_max'].astype(int)

annotations_df['image_path'] = annotations_df['image_path'].apply(lambda x: x[0])
annotations_df.head()

,image_path,x_min,y_min,x_max,y_max
0,car_ims/000001.jpg,112,7,853,717
1,car_ims/000002.jpg,48,24,441,202
2,car_ims/000003.jpg,7,4,277,180
3,car_ims/000004.jpg,33,50,197,150
4,car_ims/000005.jpg,5,8,83,58


In [ ]:
import cv2
import numpy as np

def preprocess_image(image_path):
    # Load the image from the provided path
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Resize the image to the required input shape (e.g., 224x224)
    img = cv2.resize(img, (224, 224))
    
   

    # Normalize pixel values to the range [0, 1]
    img = img.astype(np.float32) / 255.0

    return img

# Preprocess the images and create the training dataset
images = [preprocess_image(f"/kaggle/input/stanford-cars-dataset/cars_train/cars_train/{path}") for path in cars_train_files]
images = np.array(images)

# Extract the bounding box coordinates (x_min, y_min, x_max, y_max) from the annotations DataFrame
x_min = annotations_df['x_min'].values
y_min = annotations_df['y_min'].values
x_max = annotations_df['x_max'].values
y_max = annotations_df['y_max'].values

# Normalize the bounding box coordinates to the range [0, 1]
x_center = (x_min + x_max) / 2 / 224.0
y_center = (y_min + y_max) / 2 / 224.0
width = (x_max - x_min) / 224.0
height = (y_max - y_min) / 224.0

# Combine the bounding box coordinates into a single array
annotations = np.column_stack((x_center, y_center, width, height))



In [ ]:
import numpy as np

# Assuming you have already loaded the data and annotations as shown in your previous code

# Calculate the number of images you want to keep
desired_num_images = 3000  # Replace with the desired number of images

# Check the current number of images and annotations
num_images = len(images)
num_annotations = len(annotations)

# Ensure that the number of desired images is not greater than the current number of images
desired_num_images = min(desired_num_images, num_images)

# Randomly select a subset of indices for the annotations
selected_indices = np.random.choice(num_annotations, desired_num_images, replace=False)

# Create new annotations and images arrays with the selected subset
selected_images = images[:desired_num_images]
selected_annotations = annotations[selected_indices]


In [ ]:
# Assuming you have already loaded the data and annotations as shown in your previous code
# Assuming annotations have the format (x_min, y_min, x_max, y_max) for each bounding box
# You can modify this code as needed based on your specific annotation format.

# Define the number of grid cells in the x and y directions
grid_size = 7

def convert_annotations_to_yolo_format(annotations, input_shape, grid_size):
    image_width, image_height = input_shape[:2]
    grid_width = image_width / grid_size
    grid_height = image_height / grid_size

    yolo_annotations = np.zeros((len(annotations), grid_size, grid_size, 6))

    for i, (x_min, y_min, x_max, y_max) in enumerate(annotations):
        x_center = (x_min + x_max) / 2
        y_center = (y_min + y_max) / 2
        width = x_max - x_min
        height = y_max - y_min

        grid_x = int(x_center / grid_width)
        grid_y = int(y_center / grid_height)

        yolo_annotations[i, grid_y, grid_x, 0] = x_center / image_width
        yolo_annotations[i, grid_y, grid_x, 1] = y_center / image_height
        yolo_annotations[i, grid_y, grid_x, 2] = width / image_width
        yolo_annotations[i, grid_y, grid_x, 3] = height / image_height
        yolo_annotations[i, grid_y, grid_x, 4] = 1  # Confidence score, assuming all boxes are present
        yolo_annotations[i, grid_y, grid_x, 5] = 1  # Class score, assuming only one class (car)

    return yolo_annotations

# Convert annotations to YOLO format with the new grid_size

# Convert annotations to YOLO format
input_shape=(224,224,3)
yolo_annotations_train = convert_annotations_to_yolo_format(selected_annotations, input_shape,grid_size)
# Split data into training and validation sets
# Define your split ratio as needed (e.g., 80% training, 20% validation)



In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class YOLOv3(nn.Module):
    def __init__(self, num_classes=1):
        super(YOLOv3, self).__init__()
        
        # Backbone network
        self.conv1 = nn.Conv2d(3, 32, 3) 
        self.conv2 = nn.Conv2d(32, 64, 3)
        #...
        
        # Output layers
        self.conv_out1 = nn.Conv2d(1024, 1024, 3)
        self.conv_out2 = nn.Conv2d(768, 768, 3)
        self.conv_out3 = nn.Conv2d(384, 384, 3)
        
        self.pred1 = nn.Conv2d(1024, 5+num_classes, 1)
        self.pred2 = nn.Conv2d(768, 5+num_classes, 1) 
        self.pred3 = nn.Conv2d(384, 5+num_classes, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        #...
        
        out1 = self.pred1(self.conv_out1(x))
        out2 = self.pred2(self.conv_out2(x)) 
        out3 = self.pred3(self.conv_out3(x))
        
        return out1, out2, out3

# Define the new grid size
grid_size = 7

# Assuming num_classes is defined with the correct value
num_classes = 1

# Create the YOLO model with the updated grid size
yolo_model = YOLOv3(num_classes)


In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

# Assuming you have already loaded the data and annotations as shown in your previous code
x_train,x_test,y_train,y_test = train_test_split(selected_images,yolo_annotations_train,train_size=0.8,random_state=42)
# Split data into training and validation sets
# Define your split ratio as needed (e.g., 80% training, 20% validation)
print("Training data shapes:", x_train.shape, y_train.shape)
print("Validation data shapes:", x_test.shape, y_test.shape)

In [ ]:
print(x_train.dtype)
print(y_train.dtype)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=5, mode='auto', min_delta=0.001, baseline=0.7, restore_best_weights=True)


In [ ]:
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau


# Convert to float32
x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_test = y_test.astype(np.float32)

# Convert NumPy arrays to PyTorch tensors
x_train_tensor = torch.tensor(x_train)
y_train_tensor = torch.tensor(y_train)
x_test_tensor = torch.tensor(x_test)
y_test_tensor = torch.tensor(y_test)

# Create DataLoaders for training and validation data
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(x_test_tensor, y_test_tensor)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

batch_size=32


criterion = nn.MSELoss()  # Use appropriate loss function for your task
optimizer = torch.optim.Adam(yolo_model.parameters(), lr=0.001)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
yolo_model.to(device)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    yolo_model.train()
    total_loss = 0

    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        optimizer.zero_grad()
        out1, out2, out3 = yolo_model(batch_x)
        loss = criterion(out1, batch_y) + criterion(out2, batch_y) + criterion(out3, batch_y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Compute the average loss for the epoch
    avg_loss = total_loss / len(train_loader)

    # Update the learning rate using the scheduler
    scheduler.step(avg_loss)

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

In [ ]:
image=np.expand_dims(preprocess_image('/kaggle/input/stanford-cars-dataset/cars_train/cars_train/00001.jpg'),axis=0)
bounder=yolo_model.predict(image)

In [ ]:
def predict_car(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))  # Assuming the input size of the model is (224, 224)
    image = image / 255.0  # Normalize the image
    image = np.expand_dims(image, axis=0)  # Add batch dimension

    prediction =yolo_model.predict(image)
    if prediction[0, :, :, 4].max() > 0.5:
        return True
    else:
        return False
images_directory='/kaggle/input/stanford-cars-dataset/cars_test/cars_test'

# Loop through all images in the directory
for filename in os.listdir(images_directory):
    if filename.endswith('.jpg'):
        image_path = os.path.join(images_directory, filename)
        car_present = predict_car(image_path)
        print(f"Image: {filename}, Car Present: {car_present}")

        if car_present:
            print("Car detected. Stopping the loop.")
            break